In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber, FreeSurferSource # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from pandas import DataFrame, read_csv
import statsmodels.formula.api as smf

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
study_home = '/Users/SEAlab/Documents/PPM'
hcpproc_dir = study_home + '/Data'
output_dir = study_home + '/fMRIproc/longitudinal'
workflow_dir = study_home + '/Workflows'
cov_file = study_home + '/Misc/covariates.csv'
cov_dataframe = read_csv(cov_file,index_col=None)
cov_dataframe = cov_dataframe.sort_values(by='folder')

# set analysis specs
n_slices = 4
data_type = 'cortical_thickness'

# set model specifications
model = smf.ols
model_formula = 'Y ~ age'
n_terms = 1

proc_cores = 2 # number of cores of processing for the workflows

In [2]:
## Data handling nodes

# grab data
def pull_data(data_dict,data_label):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    
    file_list = sorted(glob(data_dict[data_label]))
    
    return(file_list)

selectfiles = Node(Function(input_names=['data_dict','data_label'],
                            output_names=['file_list'],
                            function=pull_data), name='selectfiles')
selectfiles.inputs.data_dict={'cortical_thickness':hcpproc_dir + '/*/MNINonLinear/fsaverage_LR32k/*.thickness.32k_fs_LR.dscalar.nii',
                             'functional_connectivity':hcpproc_dir + ''}
selectfiles.inputs.data_label = data_type

# sink data
substitutions = []
datasink = Node(DataSink(),name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

In [3]:
# create index for cifti file
def create_indexing(file_list,n_slices):
    import nibabel as nib
    import numpy as np
    from sklearn.utils import Bunch

    img1 = nib.load(file_list[0])
    Y1 = img1.get_fdata()
    N = Y1.shape[0]
    M = Y1.shape[1]
    indexes=Bunch()

    index_labels = ['slice{0}'.format(a) for a in range(0,n_slices)]
    width = int(M/n_slices)
    start=0
    end=width
    i=0
    while end<M:
        indexes[index_labels[i]]=slice(start,end)
        start+=width
        end+=width
        i+=1
    if end-width<M:
        indexes[index_labels[i]]=slice(start,M)
    indexes_list = [indexes[x] for x in indexes.keys()]
    return(indexes_list)

# pull data from the same slice for each file
def agg_data(file_list,index):
    import nibabel as nib
    import numpy as np
    from os.path import basename, abspath
    
    data_stack = []
    for file in file_list:
        img = nib.load(file)
        temp = img.get_fdata()
        temp = temp[:,index]
        temp_file = basename(file).replace('.nii','.slice.npy')
        np.save(temp_file,temp)
        data_stack.append(abspath(temp_file))
        
    return(data_stack)

# run models on data
def run_long_models(model,model_formula,data_stack,covariates,n_terms):
    import numpy as np
    from os.path import basename,abspath

    data_list = []
    for d in data_stack:
        t = np.load(d)
        t = np.expand_dims(t,axis=2)
        data_list.append(t)

    sub_data = np.concatenate(data_list, axis=2)

    m = sub_data.shape[0]
    n = sub_data.shape[1]

    p_vals = np.zeros((m,n,n_terms+1))
    t_vals = np.zeros((m,n,n_terms+1))
    beta_vals = np.zeros((m,n,n_terms+1))

    for a in range(0,m):
        for b in range(0,n):
            covariates['Y'] = sub_data[a,b,:]
            mdl = model(formula=model_formula, data=covariates).fit()
            beta_vals[a,b,:] = mdl.params
            p_vals[a,b,:] = mdl.pvalues
            t_vals[a,b,:] = mdl.tvalues

    p_file = 'p_slice.npy'
    t_file = 't_slice.npy'
    beta_file = 'beta_slice.npy'

    np.save(p_file,p_vals)
    np.save(t_file, t_vals)
    np.save(beta_file,beta_vals)

    p_slice = abspath(p_file)
    t_slice = abspath(t_file)
    beta_slice = abspath(beta_file)
    return(p_slice,beta_slice,t_slice)

# reconstitute the results into cifti files
def combine_data(p_list,beta_list,t_list,index_list,n_terms,file_list):
    import nibabel as nib
    import numpy as np
    from os.path import abspath
    
    img = nib.load(file_list[0])
    img_data = img.get_fdata()

    p_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))
    t_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))
    beta_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))
    
    print(index_list)

    for x in range(0,len(index_list)):
        p_data[:,index_list[x],:] = np.load(p_list[x])
        t_data[:,index_list[x],:] = np.load(t_list[x])
        beta_data[:,index_list[x],:] = np.load(beta_list[x])

    # get cifti header info from first file
    bm = img.header.get_axis(1)
    sc = img.header.get_axis(0)
    
    pval_cifti = []
    tstat_cifti = []
    beta_cifti = []
    
    for x in range(0,n_terms+1):
        # p-values
        temp_p = p_data[:,:,x]
        sc.name = ['pvalues']
        p_img = nib.cifti2.cifti2.Cifti2Image(temp_p,(sc,bm))
        nib.save(p_img,'model_pvalues_term{0}.dscalar.nii'.format(x))
        pval_cifti.append(abspath('model_pvalues_term{0}.dscalar.nii'.format(x)))
        # t-values
        temp_t = t_data[:,:,x]
        sc.name = ['tvalues']
        t_img = nib.cifti2.cifti2.Cifti2Image(temp_t,(sc,bm))
        nib.save(t_img,'model_tvalues_term{0}.dscalar.nii'.format(x))
        tstat_cifti.append(abspath('model_tvalues_term{0}.dscalar.nii'.format(x)))
        #beta values
        temp_b = beta_data[:,:,x]
        sc.name = ['beta']
        b_img = nib.cifti2.cifti2.Cifti2Image(temp_b,(sc,bm))
        nib.save(b_img,'model_beta_term{0}.dscalar.nii'.format(x))
        beta_cifti.append(abspath('model_beta_term{0}.dscalar.nii'.format(x)))
    
    return(pval_cifti,beta_cifti,tstat_cifti)


In [8]:
# Node to wrap the function that makes the list of indexes
make_indexes = Node(Function(input_names=['file_list','n_slices'], output_names=['indexes_list'], 
                             function=create_indexing),name='create_indexing')
make_indexes.inputs.n_slices=n_slices

# Node to wrap the function that pulls the data to model
pull_data = MapNode(Function(input_names=['file_list','index'],output_names=['data_stack'],
                             function=agg_data),name='pull_data', iterfield=['index'])

# Node to wrap the function that runs the longitudinal model
run_models = MapNode(Function(input_names=['model','model_formula','data_stack','covariates','n_terms'],
                              output_names=['p_slice','beta_slice','t_slice'], function=run_long_models),
                     name='run_models',iterfield=['data_stack'])
run_models.inputs.model=model
run_models.inputs.model_formula=model_formula
run_models.inputs.covariates=cov_dataframe
run_models.inputs.n_terms=n_terms

# Node to recombine the data slices for visualization
recombine_data = Node(Function(input_names=['p_list','beta_list','t_list','index_list','n_terms','file_list'],
                               output_names=['pval_cifti','beta_cifti','tstat_cifti'],function=combine_data),
                      name='recombine_data')
recombine_data.inputs.n_terms = n_terms

In [ ]:
long_model = Workflow(name='long_model', base_dir=workflow_dir)
long_model.connect([(selectfiles,make_indexes,[('file_list','file_list')]),
                    (selectfiles,pull_data,[('file_list','file_list')]),
                    (selectfiles,recombine_data,[('file_list','file_list')]),
                    (make_indexes, pull_data, [('indexes_list','index')]),
                    (pull_data, run_models, [('data_stack','data_stack')]),
                    (make_indexes, recombine_data, [('indexes_list','index_list')]),
                    (run_models, recombine_data, [('p_slice','p_list'),
                                                  ('beta_slice','beta_list'),
                                                  ('t_slice','t_list')]),
                    (recombine_data, datasink, [('pval_cifti','pvalues'),
                                                ('beta_cifti','betas'),
                                                ('tstat_cifti','tstats')])
                   ])
long_model.run('MultiProc', plugin_args={'n_procs':proc_cores})

In [ ]:
from glob import glob
import nibabel as nib
import numpy as np
from sklearn.utils import Bunch

file_list = sorted(glob(hcpproc_dir + '/*/MNINonLinear/fsaverage_LR32k/*.thickness.32k_fs_LR.dscalar.nii'))

img1 = nib.load(file_list[0])
Y1 = img1.get_fdata()
N = Y1.shape[0]
M = Y1.shape[1]
indexes=Bunch()

index_labels = ['slice{0}'.format(a) for a in range(0,n_slices)]
width = int(M/n_slices)
start=0
end=width
i=0
while end<M:
    indexes[index_labels[i]]=slice(start,end)
    start+=width
    end+=width
    i+=1
if end-width<M:
    indexes[index_labels[i]]=slice(start,M)
indexes_list = [indexes[x] for x in indexes.keys()]

print(Y1.shape)

In [ ]:
from glob import glob
import nibabel as nib
import numpy as np
from os.path import abspath

file_list = glob(hcpproc_dir + '/*/MNINonLinear/fsaverage_LR32k/*.thickness.32k_fs_LR.dscalar.nii')
p_list = sorted(glob('/Users/SEAlab/Documents/PPM/Workflows/long_model/run_models/mapflow/_run_models*/p_slice.npy'))
t_list = sorted(glob('/Users/SEAlab/Documents/PPM/Workflows/long_model/run_models/mapflow/_run_models*/t_slice.npy'))
beta_list = sorted(glob('/Users/SEAlab/Documents/PPM/Workflows/long_model/run_models/mapflow/_run_models*/beta_slice.npy'))

index_list = indexes_list
img = nib.load(file_list[0])
img_data = img.get_fdata()

p_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))
t_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))
beta_data = np.empty((img_data.shape[0],img_data.shape[1],n_terms+1))

for x in range(0,len(index_list)):
    p_data[:,index_list[x],:] = np.load(p_list[x])
    t_data[:,index_list[x],:] = np.load(t_list[x])
    beta_data[:,index_list[x],:] = np.load(beta_list[x])

# get cifti header info from first file
bm = img.header.get_axis(1)
sc = img.header.get_axis(0)

pval_cifti = []
tstat_cifti = []
beta_cifti = []

for x in range(0,n_terms+1):
    # p-values
    temp_p = p_data[:,:,x]
    sc.name = ['pvalues']
    p_img = nib.cifti2.cifti2.Cifti2Image(temp_p,(sc,bm))
    nib.save(p_img,'model_pvalues_term{0}.dscalar.nii'.format(x))
    pval_cifti.append(abspath('model_pvalues_term{0}.dscalar.nii'.format(x)))
    # t-values
    temp_t = t_data[:,:,x]
    sc.name = ['tvalues']
    t_img = nib.cifti2.cifti2.Cifti2Image(temp_t,(sc,bm))
    nib.save(t_img,'model_tvalues_term{0}.dscalar.nii'.format(x))
    tstat_cifti.append(abspath('model_tvalues_term{0}.dscalar.nii'.format(x)))
    #beta values
    temp_b = beta_data[:,:,x]
    sc.name = ['beta']
    b_img = nib.cifti2.cifti2.Cifti2Image(temp_b,(sc,bm))
    nib.save(b_img,'model_beta_term{0}.dscalar.nii'.format(x))
    beta_cifti.append(abspath('model_beta_term{0}.dscalar.nii'.format(x)))


In [ ]:
print(pval_cifti,beta_cifti,tstat_cifti)

In [ ]:
p_data0 = 1 - p_data[:,:,0]

p_bm = img.header.get_axis(1)
p_sc = img.header.get_axis(0)
p_sc.name = ['pvalues']
p_img = nib.cifti2.cifti2.Cifti2Image(p_data0,(p_sc,p_bm))
nib.save(p_img,'model_pvalues_term{0}.dscalar.nii'.format(x))